# Supervised Learning
So far we have looked at how we can work with numerical data in performing different types of classification.

Today we look at how machine learning can be used to process text. This is generally a field of machine learning called Natural Language Processing

<img src="images/nlp.png" height="32" width="40%" align="left">

Natural language processing has many applications such as:
    - Translation (E.g. Google translate)
    - Summarization
    - Information Retrieval: Example

<img src="images/obama.png" height="32" width="50%" align="left">

    - Sentiment Analysis

<img src="images/sentiment_analysis.png" height="42" width="50%" align="left">

    - Text Classification


<img src="images/googlenews.png" height="32" width="40%" align="left">

Before delving into an example, let's look at how text is preprocessed before being fed into a machine learning model


## Text Preprocessing
Text preprocessing generally involves a number of steps:
1. Cleaning: Removing special characters etc
2. Tokenizing. A token is a string of contiguous characters. A word is a token
3. Vectorizing: Moving from words to vectors.


In [73]:
tweets = ['This is the first tweet.', 'This is the second second tweet.','And the third one.','Is this the first tweet?'] 
#vocab = [word.lower() for tweet in tweets for word in tweet.split() ]
vocab = []
for tweet in tweets:
	for word in tweet.split():
		vocab.append(word.lower())
        
vocab = list(set(vocab))
#vocab
sorted(vocab)

['and',
 'first',
 'is',
 'one.',
 'second',
 'the',
 'third',
 'this',
 'tweet.',
 'tweet?']

### Vectorization

vectorization the general process of turning a collection of text documents into numerical feature vectors.

### CountVectorizer

Count Vectorizer is useful for tokenizing. In Sklearn tokenizing strings gives an integer id for each possible token, for instance by using white-spaces and punctuation as token separators. CountVectorizer counts the number of token occurrences i.e. the number of times a token appears


In [74]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tweets)
features = vectorizer.get_feature_names()
features

['and', 'first', 'is', 'one', 'second', 'the', 'third', 'this', 'tweet']

In [75]:
X

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [76]:
X.toarray()  

array([[0, 1, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 2, 1, 0, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 1, 1]], dtype=int64)

###  transform 
.transform changes the text to a list ot features

In [77]:
vectorizer.transform(['Lets assume this is a new tweet']).toarray()

array([[0, 0, 1, 0, 0, 0, 0, 1, 1]])

### Bi-Grams and Tri-Grams

In [78]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)

X = bigram_vectorizer.fit_transform(tweets)
features_bi = bigram_vectorizer.get_feature_names()
print(sorted(features_bi))

['and', 'and the', 'first', 'first tweet', 'is', 'is the', 'is this', 'one', 'second', 'second second', 'second tweet', 'the', 'the first', 'the second', 'the third', 'third', 'third one', 'this', 'this is', 'this the', 'tweet']


In [79]:
trigram_vectorizer = CountVectorizer(ngram_range=(1, 3),  min_df=1)

X = trigram_vectorizer.fit_transform(tweets)
features_tri = trigram_vectorizer.get_feature_names()
print(sorted(features_tri))

['and', 'and the', 'and the third', 'first', 'first tweet', 'is', 'is the', 'is the first', 'is the second', 'is this', 'is this the', 'one', 'second', 'second second', 'second second tweet', 'second tweet', 'the', 'the first', 'the first tweet', 'the second', 'the second second', 'the third', 'the third one', 'third', 'third one', 'this', 'this is', 'this is the', 'this the', 'this the first', 'tweet']


#### Problems:
1. Longer documents will have higher average count values than shorter documents
2. Some words are very common e.g. 'the', 'and', 'is' will automatically have higher counts

#### Solution:
1. Term Frequencies times Inverse Document Frequency

### TF-IDF  
Term frequency is the number of times a word appears in a document, or in this case a tweet. If the word "kenya" appears twice in a the tweet "", then the term frequency of kenya is 2.

Term Frequency can be taken as the raw count of a term but is often adjusted to account for the total number of words in the document. Example: Instead of -the word 'kenya' was used 2 times, TF says the word 'kenya' was used 20% of the time.


<img src="images/tf.png" height="32" width="40%" align="left">

To take care of the second problem, we use IDF.
Inverse Document Frequency factor is a way of diminishing the weight of terms that occur very frequently in a document set and increasing the weight of terms that occur rarely


<img src="images/idf.png" height="42" width="50%" align="left">

In [80]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X)
X_train_tfidf.shape

//anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(4, 31)

Today we will look at text processing. The overall goal will be to predict who has written some tweets.


# Author Attribution


In [81]:
##Importations
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


## Train

#### a.) Load the Data

In [82]:
#Load the data
nrs = pd.read_csv('datasets/kenya.csv')
nrs = shuffle(nrs)
nrs.head()

#Match the indices to target names
target_names = ['UKenyatta','RailaOdinga']

### b. Split the data in training and test set

In [83]:
X = nrs['tweet']

y = nrs['class']

#X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

X_train = X[:-1000]
y_train = y[:-1000]

X_test = X[-1000:]
y_test = y[-1000:]

### c.) Create and Train a classifier
#### Feature Extraction

In [84]:
#Occurences
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape



(47840, 14776)

In [85]:
#Frequencies

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

//anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(47840, 14776)

In [86]:
#Training a classifier
classifier = LogisticRegression()
clf = classifier.fit(X_train_tfidf, y_train)

### Test

In [87]:
X_tests_counts = count_vect.transform(X_test)
X_tests_tfidf = tfidf_transformer.transform(X_tests_counts)
expected  = y_test
predicted = clf.predict(X_tests_tfidf)
print("Accuracy of our model is:\n%s" % metrics.accuracy_score(expected, predicted))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Accuracy of our model is:
0.991
Confusion matrix:
[[304   9]
 [  0 687]]


//anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


### Apply

In [88]:
#Predicting Outcome
tweet1 = 'Four years ago, we made a commitment to Kenyans. We are proud that our record speaks for itself.'
tweet2 = 'Kenyans don\'t want handouts. They need a hand up initiative from their government. The economy must grow for all - not the few.'
tweet3 = 'After a successful tour of Narok county, we conclude our visit with a major rally today in Narok town.'
tweets_new = [tweet1,tweet2,tweet3]
X_new_counts = count_vect.transform(tweets_new)

X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for tw, category in zip(tweets_new, predicted):
    print('\n%r ===> %s' % (tw, my_data['target_names'][category]))


'Four years ago, we made a commitment to Kenyans. We are proud that our record speaks for itself.' ===> UKenyatta

"Kenyans don't want handouts. They need a hand up initiative from their government. The economy must grow for all - not the few." ===> RailaOdinga

'After a successful tour of Narok county, we conclude our visit with a major rally today in Narok town.' ===> RailaOdinga


//anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
